# 서울시 편의점 매출 결정요인 분석---## Executive Summary본 프로젝트는 서울시 편의점 매출을 결정하는 핵심 요인을 데이터 기반으로 분석하여,  **신규 출점 입지 선정** 및 **기존 점포 운영 전략** 수립에 활용할 수 있는 인사이트를 도출합니다.### 핵심 발견- **상권유형**이 매출에 가장 큰 영향 (관광특구 > 발달상권 > 전통시장 > 골목상권)- **점포수**와 **유동인구**가 매출과 강한 양의 상관관계- 머신러닝 모델(RandomForest)이 매출 예측에서 R²=0.86 달성### 분석 흐름```1. 프로젝트 개요 → 2. 데이터 소개 → 3. EDA/가설검증 → 4. 회귀분석 → 5. 머신러닝 예측 → 6. What-if 시나리오 → 7. 인사이트 → 8. 결론/제언```---

# 1. 프로젝트 개요## 1.1 주제 선정 이유편의점 출점·운영 판단이 현장에서는 **"사람 많아 보인다"** 같은 경험/감각에 의존하는 경우가 많습니다.  그런데 실제 매출은 유동인구 외에도 점포 밀집도, 상권 성격 등 여러 요인이 동시에 작동하므로,  **무엇이 더 중요한지 정량 근거로 설명할 필요**가 있습니다.> 본 분석은 데이터 기반으로 매출 결정 요인을 검증하고,  > 신규 출점 및 기존 점포 운영에 활용할 수 있는 인사이트를 도출합니다.## 1.2 연구 질문**"서울시 편의점 매출을 결정하는 핵심 요인은 무엇인가?"**## 1.3 가설 설정| 가설 | 내용 | 검증방법 ||------|------|----------|| **H1** | 유동인구가 많을수록 매출이 높다 | 상관분석, 회귀분석 || **H2** | 점포수(밀집도)가 높을수록 매출이 높다 | 상관분석, 회귀분석 || **H3** | 상권유형은 매출에 유의한 영향이 있다 | 회귀분석 (더미변수) || **H4** | 상권유형 간 평균 매출 차이가 있다 | ANOVA + Tukey HSD |> ※ **사전 점검**: 유동인구 ↔ 점포수 간 다중공선성 → VIF 분석으로 확인---

# 2. 환경 설정 및 데이터 로드

In [ ]:
# 라이브러리 임포트import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom scipy import statsimport warningswarnings.filterwarnings('ignore')# 통계/회귀 분석import statsmodels.api as smfrom statsmodels.stats.outliers_influence import variance_inflation_factorfrom statsmodels.stats.diagnostic import het_breuschpaganfrom statsmodels.stats.multicomp import pairwise_tukeyhsd# 머신러닝from sklearn.model_selection import train_test_split, cross_val_scorefrom sklearn.metrics import r2_score, mean_absolute_error, mean_squared_errorfrom sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor# 한글 폰트 설정from matplotlib import font_managercandidates = ["Malgun Gothic", "AppleGothic", "NanumGothic", "Noto Sans CJK KR"]available = {f.name for f in font_manager.fontManager.ttflist}for c in candidates:    if c in available:        plt.rcParams["font.family"] = c        breakplt.rcParams["axes.unicode_minus"] = Falseplt.rcParams['figure.figsize'] = (10, 6)# outputs 폴더 생성import osos.makedirs('./outputs', exist_ok=True)print("환경 설정 완료 ✓")

In [ ]:
# 데이터 로드df = pd.read_csv('분석데이터.csv', encoding='utf-8')print(f"데이터 로드 완료: {len(df):,}건")print(f"기간: {df['기준_년분기_코드'].min()} ~ {df['기준_년분기_코드'].max()}")print(f"분기 수: {df['기준_년분기_코드'].nunique()}개")

# 3. 데이터 소개## 3.1 데이터 출처 및 구조

In [ ]:
print("=" * 60)print("데이터 개요")print("=" * 60)print(f"""[데이터 출처]  - 서울 열린데이터광장 (data.seoul.go.kr)  - 서울시 우리마을가게 상권분석서비스  [분석 단위]  - 기준년분기 × 행정동 (Raw 데이터를 Group By 집계)  [분석 기간]  - 2022년 1분기 ~ 2025년 3분기 ({df['기준_년분기_코드'].nunique()}개 분기)  [데이터 규모]  - 행: {len(df):,}건  - 열: {len(df.columns)}개""")print("\n[변수 정의]")print("-" * 60)var_desc = {    '기준_년분기_코드': '분석 기준 년도+분기 (예: 20221 = 2022년 1분기)',    '행정동_코드_명': '행정동 이름',    '당월_매출_금액': '해당 분기 편의점 총 매출액 (원) - [종속변수 Y]',    '점포_수': '해당 행정동 편의점 수 - [독립변수 X1]',    '총_유동인구_수': '해당 행정동 총 유동인구 - [독립변수 X2]',    '주요_상권유형': '상권 분류 - [독립변수 X3]',}for col, desc in var_desc.items():    print(f"  {col}: {desc}")

In [ ]:
# 데이터 미리보기print("\n[데이터 미리보기]")df.head()

## 3.2 데이터 품질 검증

In [ ]:
print("=" * 60)print("데이터 품질 검증")print("=" * 60)# 결측치 확인print("\n[1] 결측치 현황")missing = df.isna().sum()if missing.sum() == 0:    print("  → 결측치 없음 ✓")else:    print(missing[missing > 0])# 중복행 확인print(f"\n[2] 중복행: {df.duplicated().sum()}건")# 상권유형별 샘플 수print("\n[3] 상권유형별 샘플 수")print(df['주요_상권유형'].value_counts())

---# 4. 탐색적 데이터 분석 (EDA) 및 가설 검증## 4.1 종속변수 탐색: 매출 분포

In [ ]:
sales = df['당월_매출_금액']print("=" * 60)print("종속변수(Y): 당월 매출 금액")print("=" * 60)print(f'''[기초 통계량]  평균: {sales.mean()/1e8:.2f}억원  중앙값: {sales.median()/1e8:.2f}억원  표준편차: {sales.std()/1e8:.2f}억원  최소: {sales.min()/1e8:.2f}억원  최대: {sales.max()/1e8:.2f}억원[분포 특성]  왜도(Skewness): {sales.skew():.2f}  첨도(Kurtosis): {sales.kurtosis():.2f}''')if sales.skew() > 1:    print("  → 오른쪽 꼬리가 긴 분포 (고매출 지역이 소수 존재)")    print("  → 이분산(Heteroscedasticity) 가능성 → Robust SE 사용 권장")

In [ ]:
# 매출 분포 시각화fig, axes = plt.subplots(1, 2, figsize=(14, 5))# 원본 분포axes[0].hist(df['당월_매출_금액']/1e8, bins=50, color='steelblue', edgecolor='white', alpha=0.7)axes[0].axvline(sales.mean()/1e8, color='red', linestyle='--', linewidth=2, label=f'평균: {sales.mean()/1e8:.1f}억')axes[0].axvline(sales.median()/1e8, color='orange', linestyle='--', linewidth=2, label=f'중앙값: {sales.median()/1e8:.1f}억')axes[0].set_xlabel('매출 (억원)', fontsize=12)axes[0].set_ylabel('빈도', fontsize=12)axes[0].set_title('매출 분포 (원본)', fontsize=14, fontweight='bold')axes[0].legend()# 로그 변환 분포axes[1].hist(np.log1p(df['당월_매출_금액']), bins=50, color='darkgreen', edgecolor='white', alpha=0.7)axes[1].set_xlabel('log(매출)', fontsize=12)axes[1].set_ylabel('빈도', fontsize=12)axes[1].set_title('매출 분포 (로그 변환)', fontsize=14, fontweight='bold')plt.tight_layout()plt.savefig('./outputs/01_매출분포.png', dpi=150, bbox_inches='tight')plt.show()

## 4.2 가설 H1 검증: 유동인구 → 매출**H1: 유동인구가 많을수록 매출이 높다**

In [ ]:
# 상관관계 분석corr_h1, pval_h1 = stats.pearsonr(df['총_유동인구_수'], df['당월_매출_금액'])print("=" * 60)print("H1 검증: 유동인구 → 매출")print("=" * 60)print(f'''[상관분석 결과]  Pearson 상관계수: r = {corr_h1:.4f}  p-value: {pval_h1:.2e}  [결론] H1 채택: 유동인구가 많을수록 매출이 높다 (p < 0.001)''')

In [ ]:
# H1 시각화fig, ax = plt.subplots(figsize=(10, 6))ax.scatter(df['총_유동인구_수']/1e6, df['당월_매출_금액']/1e8, alpha=0.3, s=20, color='steelblue')# 추세선z = np.polyfit(df['총_유동인구_수'], df['당월_매출_금액'], 1)p = np.poly1d(z)x_line = np.linspace(df['총_유동인구_수'].min(), df['총_유동인구_수'].max(), 100)ax.plot(x_line/1e6, p(x_line)/1e8, color='red', linewidth=2, label='추세선')ax.text(0.05, 0.95, f'r = {corr_h1:.3f}\np < 0.001', transform=ax.transAxes,         fontsize=12, verticalalignment='top',         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))ax.set_xlabel('유동인구 (백만명)', fontsize=12)ax.set_ylabel('매출 (억원)', fontsize=12)ax.set_title('H1 검증: 유동인구와 매출의 관계', fontsize=14, fontweight='bold')ax.legend()ax.grid(alpha=0.3)plt.tight_layout()plt.savefig('./outputs/02_H1_유동인구_매출.png', dpi=150, bbox_inches='tight')plt.show()

## 4.3 가설 H2 검증: 점포수 → 매출**H2: 점포수(밀집도)가 높을수록 매출이 높다**> ⚠️ **주의**: 역인과 가능성 존재 (수요가 있는 곳에 점포가 많이 생김)

In [ ]:
# 상관관계 분석corr_h2, pval_h2 = stats.pearsonr(df['점포_수'], df['당월_매출_금액'])print("=" * 60)print("H2 검증: 점포수 → 매출")print("=" * 60)print(f'''[상관분석 결과]  Pearson 상관계수: r = {corr_h2:.4f}  p-value: {pval_h2:.2e}  [결론] H2 채택: 점포수가 많을수록 매출이 높다 (p < 0.001)[주의] 상관관계 ≠ 인과관계 (역인과 가능성)''')

In [ ]:
# H2 시각화fig, ax = plt.subplots(figsize=(10, 6))ax.scatter(df['점포_수'], df['당월_매출_금액']/1e8, alpha=0.3, s=20, color='darkgreen')z = np.polyfit(df['점포_수'], df['당월_매출_금액'], 1)p = np.poly1d(z)x_line = np.linspace(df['점포_수'].min(), df['점포_수'].max(), 100)ax.plot(x_line, p(x_line)/1e8, color='red', linewidth=2, label='추세선')ax.text(0.05, 0.95, f'r = {corr_h2:.3f}\np < 0.001', transform=ax.transAxes,         fontsize=12, verticalalignment='top',         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))ax.set_xlabel('점포 수 (개)', fontsize=12)ax.set_ylabel('매출 (억원)', fontsize=12)ax.set_title('H2 검증: 점포수와 매출의 관계', fontsize=14, fontweight='bold')ax.legend()ax.grid(alpha=0.3)plt.tight_layout()plt.savefig('./outputs/03_H2_점포수_매출.png', dpi=150, bbox_inches='tight')plt.show()

## 4.4 사전 점검: 다중공선성 (유동인구 ↔ 점포수)

In [ ]:
# 독립변수 간 상관관계corr_x1x2, pval_x1x2 = stats.pearsonr(df['총_유동인구_수'], df['점포_수'])print("=" * 60)print("사전 점검: 다중공선성")print("=" * 60)print(f'''[유동인구 ↔ 점포수 상관계수]  r = {corr_x1x2:.4f}[판단]  r < 0.7 → 다중공선성 문제 낮음 ✓  → VIF 분석으로 최종 확인 예정''')

## 4.5 가설 H4 검증: 상권유형 간 매출 차이 (ANOVA)**H4: 상권유형 간 평균 매출 차이가 있다**

In [ ]:
# 상권유형별 통계print("=" * 60)print("H4 검증: 상권유형별 매출 차이")print("=" * 60)type_stats = df.groupby('주요_상권유형')['당월_매출_금액'].agg(['mean', 'median', 'count'])type_stats['mean_억'] = type_stats['mean'] / 1e8type_stats = type_stats.sort_values('mean', ascending=False)print("\n[상권유형별 매출 현황]")for idx, row in type_stats.iterrows():    print(f"  {idx}: 평균 {row['mean_억']:.2f}억원 (n={row['count']:.0f})")

In [ ]:
# ANOVA 검정groups = [group['당월_매출_금액'].values for name, group in df.groupby('주요_상권유형')]f_stat, anova_pval = stats.f_oneway(*groups)print(f'''[ANOVA 검정 결과]  F-통계량: {f_stat:.2f}  p-value: {anova_pval:.2e}  [결론] H4 채택: 상권유형 간 매출 차이가 유의함 (p < 0.001)''')# 골목상권 대비 차이base_mean = df[df['주요_상권유형'] == '골목상권']['당월_매출_금액'].mean()print("[골목상권 대비 매출 차이]")for t in ['관광특구', '발달상권', '전통시장', '미분류']:    if t in df['주요_상권유형'].values:        t_mean = df[df['주요_상권유형'] == t]['당월_매출_금액'].mean()        diff = (t_mean - base_mean) / 1e8        print(f"  {t}: {diff:+.2f}억원")

In [ ]:
# H4 시각화: 상권유형별 박스플롯fig, ax = plt.subplots(figsize=(12, 6))order = type_stats.index.tolist()colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']sns.boxplot(data=df, x='주요_상권유형', y='당월_매출_금액', order=order, palette=colors, ax=ax)ax.set_ylabel('매출 (원)', fontsize=12)ax.set_xlabel('상권유형', fontsize=12)ax.set_title(f'H4 검증: 상권유형별 매출 분포 (ANOVA F={f_stat:.1f}, p<0.001)', fontsize=14, fontweight='bold')ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e8:.0f}억'))ax.grid(alpha=0.3, axis='y')plt.tight_layout()plt.savefig('./outputs/04_H4_상권유형별_매출.png', dpi=150, bbox_inches='tight')plt.show()

## 4.6 상관관계 히트맵

In [ ]:
# 수치형 변수 상관관계numeric_cols = ['당월_매출_금액', '당월_매출_건수', '점포_수', '총_유동인구_수']corr_matrix = df[numeric_cols].corr()fig, ax = plt.subplots(figsize=(8, 6))sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='RdYlBu_r', center=0,            square=True, linewidths=0.5, ax=ax, annot_kws={'size': 12})ax.set_title('변수 간 상관관계 히트맵', fontsize=14, fontweight='bold')plt.tight_layout()plt.savefig('./outputs/05_상관관계_히트맵.png', dpi=150, bbox_inches='tight')plt.show()

## 4.7 가설 검증 종합 (EDA 단계)

In [ ]:
print("=" * 70)print("가설 검증 종합 (EDA 단계)")print("=" * 70)print(f"""┌────────┬──────────────────────────────────┬──────────┬─────────────────┐│  가설  │               내용               │ 예비결과 │      근거       │├────────┼──────────────────────────────────┼──────────┼─────────────────┤│   H1   │ 유동인구↑ → 매출↑                │  지지    │ r={corr_h1:.3f}        │├────────┼──────────────────────────────────┼──────────┼─────────────────┤│   H2   │ 점포수↑ → 매출↑                  │  지지    │ r={corr_h2:.3f}        │├────────┼──────────────────────────────────┼──────────┼─────────────────┤│   H3   │ 상권유형 → 매출 영향             │  지지    │ 회귀분석 확인   │├────────┼──────────────────────────────────┼──────────┼─────────────────┤│   H4   │ 상권유형 간 매출 차이            │  지지    │ F={f_stat:.1f}         │└────────┴──────────────────────────────────┴──────────┴─────────────────┘""")

---# 5. 회귀분석## 5.1 변수 준비

In [ ]:
# 더미변수 생성 (골목상권을 기준범주로)df['발달상권'] = (df['주요_상권유형'] == '발달상권').astype(int)df['전통시장'] = (df['주요_상권유형'] == '전통시장').astype(int)df['관광특구'] = (df['주요_상권유형'] == '관광특구').astype(int)df['미분류'] = (df['주요_상권유형'] == '미분류').astype(int)X = df[['점포_수', '총_유동인구_수', '발달상권', '전통시장', '관광특구', '미분류']]y = df['당월_매출_금액']print("[회귀분석 변수 설정]")print(f"  종속변수(Y): 당월_매출_금액")print(f"  독립변수(X): {list(X.columns)}")print(f"  기준범주: 골목상권")print(f"  관측치 수: {len(X):,}개")

## 5.2 다중공선성 검정 (VIF)

In [ ]:
# VIF 계산X_const = sm.add_constant(X)vif_data = []for i, col in enumerate(X_const.columns):    if col != 'const':        vif_val = variance_inflation_factor(X_const.values, i)        vif_data.append({'변수': col, 'VIF': vif_val})vif_df = pd.DataFrame(vif_data).sort_values('VIF', ascending=False)print("=" * 50)print("다중공선성 검정 (VIF)")print("=" * 50)print("\n[VIF 결과]")print(vif_df.to_string(index=False))print(f'''[해석 기준]  VIF < 5: 문제 없음 ✓  VIF 5~10: 주의 ⚠️  VIF > 10: 심각 ❌  [결론] 최대 VIF = {vif_df['VIF'].max():.2f} → 다중공선성 문제 없음 ✓''')vif_df.to_csv('./outputs/VIF.csv', index=False, encoding='utf-8-sig')

## 5.3 OLS 회귀분석 (H1, H2, H3 최종 검증)

In [ ]:
# OLS 회귀분석 (HC3 Robust SE)X_const = sm.add_constant(X).astype(float)ols_model = sm.OLS(y.values, X_const.values).fit(cov_type='HC3')print("=" * 70)print("OLS 회귀분석 결과 (Robust SE - HC3)")print("=" * 70)print(f'''[모델 적합도]  R² = {ols_model.rsquared:.4f} (설명력: {ols_model.rsquared*100:.1f}%)  Adj. R² = {ols_model.rsquared_adj:.4f}  F-statistic = {ols_model.fvalue:.2f} (p = {ols_model.f_pvalue:.2e})''')

In [ ]:
# 회귀계수 정리coef_df = pd.DataFrame({    '변수': X_const.columns,    '계수': ols_model.params,    't값': ols_model.tvalues,    'p값': ols_model.pvalues})coef_df['계수(억원)'] = coef_df['계수'] / 1e8coef_df['유의성'] = coef_df['p값'].apply(lambda p: '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else '')print("\n[회귀계수]")print(coef_df[['변수', '계수(억원)', 't값', 'p값', '유의성']].to_string(index=False))print("\n※ 유의수준: *** p<0.001, ** p<0.01, * p<0.05")coef_df.to_csv('./outputs/OLS_HC3_계수.csv', index=False, encoding='utf-8-sig')

## 5.4 잔차 진단

In [ ]:
# 잔차 계산fitted = ols_model.fittedvaluesresid = ols_model.resid# Breusch-Pagan 이분산 검정bp_test = het_breuschpagan(resid, X_const.values)bp_pval = bp_test[1]print("=" * 50)print("잔차 진단: 이분산 검정 (Breusch-Pagan)")print("=" * 50)print(f'''[검정 결과]  LM 통계량: {bp_test[0]:.2f}  p-value: {bp_pval:.2e}  [결론] 이분산 존재 → Robust SE(HC3) 사용이 적절함 ✓''')

In [ ]:
# 잔차 시각화fig, axes = plt.subplots(1, 2, figsize=(14, 5))# 잔차 vs 예측값axes[0].scatter(fitted/1e8, resid/1e8, alpha=0.3, s=10)axes[0].axhline(y=0, color='red', linestyle='--', linewidth=1.5)axes[0].set_xlabel('예측값 (억원)', fontsize=12)axes[0].set_ylabel('잔차 (억원)', fontsize=12)axes[0].set_title('잔차 vs 예측값', fontsize=14, fontweight='bold')axes[0].grid(alpha=0.3)# QQ Plotsm.qqplot(resid, line='45', ax=axes[1], alpha=0.3)axes[1].set_title('Q-Q Plot', fontsize=14, fontweight='bold')axes[1].grid(alpha=0.3)plt.tight_layout()plt.savefig('./outputs/06_잔차진단.png', dpi=150, bbox_inches='tight')plt.show()

## 5.5 사후검정 (Tukey HSD)

In [ ]:
# Tukey HSD 사후검정tukey = pairwise_tukeyhsd(df['당월_매출_금액'], df['주요_상권유형'], alpha=0.05)print("=" * 70)print("Tukey HSD 사후검정")print("=" * 70)print(tukey.summary())tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])tukey_df.to_csv('./outputs/TukeyHSD.csv', index=False, encoding='utf-8-sig')

---# 6. 머신러닝 예측 모델## 6.1 모델 학습 및 평가

In [ ]:
# Train/Test 분리X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)print(f"학습 데이터: {len(X_train):,}건")print(f"테스트 데이터: {len(X_test):,}건")

In [ ]:
# 모델 학습results = []# Linear RegressionX_train_c = sm.add_constant(X_train).astype(float)X_test_c = sm.add_constant(X_test, has_constant='add').astype(float)lr_model = sm.OLS(y_train.values, X_train_c.values).fit()pred_lr = lr_model.predict(X_test_c.values)results.append({'모델': 'Linear (OLS)', 'Test R²': r2_score(y_test, pred_lr),                 'Test MAE': mean_absolute_error(y_test, pred_lr)})# Random Forestrf_model = RandomForestRegressor(n_estimators=250, random_state=42, n_jobs=-1,                                   min_samples_leaf=2, oob_score=True)rf_model.fit(X_train, y_train)pred_rf = rf_model.predict(X_test)results.append({'모델': 'Random Forest', 'Test R²': r2_score(y_test, pred_rf),                'Test MAE': mean_absolute_error(y_test, pred_rf), 'OOB R²': rf_model.oob_score_})# Gradient Boostinggbr_model = GradientBoostingRegressor(n_estimators=100, random_state=42, max_depth=5)gbr_model.fit(X_train, y_train)pred_gbr = gbr_model.predict(X_test)cv_scores = cross_val_score(gbr_model, X_train, y_train, cv=5, scoring='r2')results.append({'모델': 'Gradient Boosting', 'Test R²': r2_score(y_test, pred_gbr),                'Test MAE': mean_absolute_error(y_test, pred_gbr), 'CV R²': cv_scores.mean()})results_df = pd.DataFrame(results)results_df['Test MAE(억)'] = results_df['Test MAE'] / 1e8print("=" * 70)print("모델 성능 비교")print("=" * 70)print(results_df[['모델', 'Test R²', 'Test MAE(억)']].to_string(index=False))print(f'''[결론]  - Random Forest가 Test R²={r2_score(y_test, pred_rf):.4f}로 최고 성능  - 해석: Linear (OLS), 예측: Random Forest 권장''')results_df.to_csv('./outputs/모델성능_비교.csv', index=False, encoding='utf-8-sig')

## 6.2 Feature Importance

In [ ]:
# Feature Importanceimportance_df = pd.DataFrame({    '변수': X.columns,    '중요도': rf_model.feature_importances_}).sort_values('중요도', ascending=True)fig, ax = plt.subplots(figsize=(10, 6))bars = ax.barh(importance_df['변수'], importance_df['중요도'], color='steelblue')ax.set_xlabel('Feature Importance', fontsize=12)ax.set_title('Random Forest - Feature Importance', fontsize=14, fontweight='bold')ax.grid(alpha=0.3, axis='x')for bar, val in zip(bars, importance_df['중요도']):    ax.text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.3f}', va='center', fontsize=10)plt.tight_layout()plt.savefig('./outputs/07_RF_Feature_Importance.png', dpi=150, bbox_inches='tight')plt.show()importance_df.to_csv('./outputs/RF_중요도.csv', index=False, encoding='utf-8-sig')

## 6.3 예측 vs 실제

In [ ]:
# 예측 vs 실제 산점도fig, axes = plt.subplots(1, 3, figsize=(16, 5))models = [('Linear', pred_lr), ('Random Forest', pred_rf), ('Gradient Boosting', pred_gbr)]for ax, (name, pred) in zip(axes, models):    r2 = r2_score(y_test, pred)    ax.scatter(y_test/1e8, pred/1e8, alpha=0.3, s=20)    max_val = max(y_test.max(), pred.max()) / 1e8    ax.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='완벽한 예측')    ax.set_xlabel('실제 매출 (억원)', fontsize=11)    ax.set_ylabel('예측 매출 (억원)', fontsize=11)    ax.set_title(f'{name}\nR²={r2:.4f}', fontsize=12, fontweight='bold')    ax.legend()    ax.grid(alpha=0.3)plt.tight_layout()plt.savefig('./outputs/08_예측vs실제_비교.png', dpi=150, bbox_inches='tight')plt.show()

---# 7. What-if 시나리오 분석

In [ ]:
# 기준 조건mean_stores = df['점포_수'].mean()mean_pop = df['총_유동인구_수'].mean()base_X = np.array([[1, mean_stores, mean_pop, 0, 0, 0, 0]])base_pred = lr_model.predict(base_X)[0]print("=" * 50)print("What-if 시나리오")print("=" * 50)print(f'''[기준 조건] 평균적인 골목상권  - 점포 수: {mean_stores:.1f}개  - 유동인구: {mean_pop/1e6:.2f}백만명  - 기준 예측 매출: {base_pred/1e8:.2f}억원''')# 상권유형 전환 효과print("[상권유형 전환 효과]")type_dummies = {'골목상권': [0,0,0,0], '발달상권': [1,0,0,0], '전통시장': [0,1,0,0],                 '관광특구': [0,0,1,0], '미분류': [0,0,0,1]}for type_name, dummies in type_dummies.items():    new_X = np.array([[1, mean_stores, mean_pop] + dummies])    new_pred = lr_model.predict(new_X)[0]    change = new_pred - base_pred    print(f"  {type_name}: {new_pred/1e8:.2f}억원 (골목 대비 {change/1e8:+.2f}억)")

---# 8. 비즈니스 인사이트## 8.1 저평가/고평가 지역 분석

In [ ]:
# 예측값 및 잔차 계산X_full = sm.add_constant(X).astype(float)df['예측_매출'] = lr_model.predict(X_full.values)df['잔차'] = df['당월_매출_금액'] - df['예측_매출']# 최근 분기 데이터latest_qtr = df['기준_년분기_코드'].max()df_latest = df[df['기준_년분기_코드'] == latest_qtr].copy()print(f"분석 기준: {latest_qtr//10}년 {latest_qtr%10}분기 ({len(df_latest)}개 행정동)")

In [ ]:
# 저평가 지역 TOP 10print("=" * 70)print("저평가 지역 TOP 10 (잠재력 높은 지역)")print("=" * 70)underperform = df_latest.nsmallest(10, '잔차')[['행정동_코드_명', '주요_상권유형', '점포_수',                                                 '당월_매출_금액', '예측_매출', '잔차']].copy()underperform['실제(억)'] = underperform['당월_매출_금액'] / 1e8underperform['예측(억)'] = underperform['예측_매출'] / 1e8underperform['차이(억)'] = underperform['잔차'] / 1e8for _, row in underperform.iterrows():    print(f"  {row['행정동_코드_명']:<12} {row['주요_상권유형']:<8} 실제:{row['실제(억)']:>6.1f}억 예측:{row['예측(억)']:>6.1f}억 차이:{row['차이(억)']:>+6.1f}억")

In [ ]:
# 고평가 지역 TOP 10print("\n" + "=" * 70)print("고평가 지역 TOP 10 (벤치마킹 대상)")print("=" * 70)outperform = df_latest.nlargest(10, '잔차')[['행정동_코드_명', '주요_상권유형', '점포_수',                                             '당월_매출_금액', '예측_매출', '잔차']].copy()outperform['실제(억)'] = outperform['당월_매출_금액'] / 1e8outperform['예측(억)'] = outperform['예측_매출'] / 1e8outperform['차이(억)'] = outperform['잔차'] / 1e8for _, row in outperform.iterrows():    print(f"  {row['행정동_코드_명']:<12} {row['주요_상권유형']:<8} 실제:{row['실제(억)']:>6.1f}억 예측:{row['예측(억)']:>6.1f}억 차이:{row['차이(억)']:>+6.1f}억")

In [ ]:
# 저평가/고평가 시각화fig, ax = plt.subplots(figsize=(12, 8))ax.scatter(df_latest['예측_매출']/1e8, df_latest['당월_매출_금액']/1e8,            alpha=0.4, s=30, c='gray', label='일반 지역')ax.scatter(underperform['예측(억)'], underperform['실제(억)'],            c='red', s=100, marker='v', label='저평가 TOP 10', zorder=5)ax.scatter(outperform['예측(억)'], outperform['실제(억)'],            c='blue', s=100, marker='^', label='고평가 TOP 10', zorder=5)max_val = max(df_latest['예측_매출'].max(), df_latest['당월_매출_금액'].max()) / 1e8ax.plot([0, max_val], [0, max_val], 'k--', linewidth=2, label='예측=실제')ax.set_xlabel('예측 매출 (억원)', fontsize=12)ax.set_ylabel('실제 매출 (억원)', fontsize=12)ax.set_title('저평가/고평가 지역 식별', fontsize=14, fontweight='bold')ax.legend()ax.grid(alpha=0.3)plt.tight_layout()plt.savefig('./outputs/09_저평가_고평가_지역.png', dpi=150, bbox_inches='tight')plt.show()

## 8.2 핵심 인사이트 정리

In [ ]:
print("=" * 70)print("핵심 인사이트")print("=" * 70)print("""[인사이트 1] 상권유형이 매출의 핵심 결정 요인  - 관광특구: 골목상권 대비 +27억원  - 발달상권: 골목상권 대비 +15억원[인사이트 2] 저평가 지역 = 성장 기회  - 조건 대비 매출이 낮은 지역 다수 존재  - 운영 개선 또는 신규 진입 기회[인사이트 3] 상관관계 ≠ 인과관계  - 점포수와 매출의 강한 상관 (r=0.81)  - 그러나 역인과 가능성 존재""")

---# 9. 결론 및 제언## 9.1 가설 검증 최종 결과

In [ ]:
print("=" * 70)print("가설 검증 최종 결과")print("=" * 70)print(f"""┌────────┬──────────────────────────────────┬────────┐│  가설  │               내용               │  결과  │├────────┼──────────────────────────────────┼────────┤│   H1   │ 유동인구↑ → 매출↑                │  채택  │├────────┼──────────────────────────────────┼────────┤│   H2   │ 점포수↑ → 매출↑                  │  채택  │├────────┼──────────────────────────────────┼────────┤│   H3   │ 상권유형 → 매출 영향             │  채택  │├────────┼──────────────────────────────────┼────────┤│   H4   │ 상권유형 간 매출 차이            │  채택  │└────────┴──────────────────────────────────┴────────┘[모델 성능]  - OLS R² = {ols_model.rsquared:.4f}  - Random Forest R² = {r2_score(y_test, pred_rf):.4f}""")

## 9.2 실무적 제언

In [ ]:
print("=" * 70)print("실무적 제언")print("=" * 70)print("""[신규 출점]  1. 상권유형 우선 확인 (관광특구 > 발달상권 > 전통시장 > 골목상권)  2. 유동인구 데이터 활용  3. 점포 밀집 지역은 경쟁 강도 고려[기존 점포]  - 발달상권/관광특구: 충동구매 상품, 프리미엄  - 골목상권: 생활밀착형, 배달서비스  - 미분류: 야간/새벽 틈새, 부가서비스""")

## 9.3 분석의 한계

In [ ]:
print("=" * 70)print("분석의 한계")print("=" * 70)print("""[1] 변수 한계: 임대료, 경쟁강도, 브랜드 등 미포함[2] 분석 단위: 행정동 집계 (개별 점포 수준 아님)[3] 인과관계 주의: 상관관계 ≠ 인과관계 (역인과 가능성)[4] 시계열 특성: 패널 고정효과 미적용""")

## 9.4 최종 요약

In [ ]:
print("=" * 70)print("최종 요약")print("=" * 70)print("""╔═════════════════════════════════════════════════════════════════╗║              서울시 편의점 매출 분석 결론                       ║╠═════════════════════════════════════════════════════════════════╣║  [핵심 발견]                                                    ║║    1. 상권유형 > 점포수 > 유동인구 순으로 매출에 영향           ║║    2. 관광특구/발달상권이 골목상권 대비 현저히 높은 매출        ║║    3. 모든 가설(H1~H4) 통계적으로 유의하게 채택                 ║║                                                                 ║║  [주의사항]                                                     ║║    • 상관관계 ≠ 인과관계                                        ║║    • 실제 의사결정 시 현장 검증 필수                            ║╚═════════════════════════════════════════════════════════════════╝""")print("\n분석 완료. 감사합니다.")

---## 생성된 파일 목록

In [ ]:
import osprint("=" * 50)print("생성된 결과 파일")print("=" * 50)if os.path.exists('./outputs'):    output_files = os.listdir('./outputs')    for f in sorted(output_files):        size = os.path.getsize(f'./outputs/{f}')        print(f"  ./outputs/{f} ({size:,} bytes)")